## Notebook to research diabetes onset 


e) Matched Controls Diabetes Onset Curve(Blue 3)

   1. Get all the canary patients who have their CC_DIABETES flag equals 0
   on their registered date.
   
       1. Pick a matching algorithm
   2. Create the macthed population
       2. For each canary patient let their registration month be i
       2. Find N(for example 50)  control patients to match this canary patient from the pmpm 
       patients on month i who have CC_DIABETES = 0 on month i. For example the N patients should have the same age 
       and then be the nearest neighbors according to other features(log costs, cc codes etc).
       2. Set the index date for these pmpm controls patients to be month i.
       2. This forms a matched set of roughly 30 times the number of canary patients and each control has an index date.
   
   3. Compute Onset of Diabetes for month i: 
       3. Get all the matched controls with index date prior to month i.
       3. For Month i we define the total number of Diabetes Onsets(DO) as the number of subjects from e.3 that had
        the flag CC_DIABETES = 1 at Month (i) and CC_DIABETES = 0 at Month (i-1) (Note these patients must be in pmpm on both months).
       3. S is the total number of matched controls included in the pmpm both in month i and month i-1 
       3. For Month i we define the diabetes onset rate(DOR) as:
             DOR =  DO/S*1000

       3. Suggestion: Remove from  S the subjects that  have the flag CC_DIABETES = 1 at month i-1

   

In [5]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [6]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Load Pmpm Canary data

In [7]:
%%time
canary_p = '../../data/CANARY/'
pmpm_p = '../../temp/CC_PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)

CPU times: user 5.46 s, sys: 6.68 s, total: 12.1 s
Wall time: 12 s


### IP-ER

In [8]:
def ip_er(group):
    x = np.arange(len(group))+1
    group['AVG_IP'] = (group.ALLOW_IP > 0).cumsum()/x
    group['AVG_ER'] = (group.ALLOW_ER > 0).cumsum()/x
    return group

def ip_er2(group, time):
    x = min(len(group), time)
    c1 = group.IP_BIN.cumsum()
    c2 = group.ER_BIN.cumsum()
    group.loc[:,'IP_BIN2'] = c1.values-c1.shift(periods = x, fill_value =0).values
    group.loc[:,'ER_BIN2'] = c2.values-c2.shift(periods = x, fill_value =0).values
    group['IP_C'] = c1
    group['ER_C'] = c2
    return group

In [9]:
%%time
pmpm = pmpm.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)
pmpm['IP_BIN'] = (pmpm.ALLOW_IP > 0).astype(int)
pmpm['ER_BIN'] = (pmpm.ALLOW_ER > 0).astype(int)

CPU times: user 6.88 s, sys: 2.54 s, total: 9.41 s
Wall time: 9.41 s


In [10]:
%%time
pmpm = pmpm.groupby('PERS_ID').apply(ip_er2, time = 6).reset_index(drop = True)

CPU times: user 31min 37s, sys: 20.2 s, total: 31min 57s
Wall time: 31min 55s


## Log the cost at source - PMPM

In [11]:
%%time
pmpm.ALLOW_AMT = pmpm.ALLOW_AMT.apply(data_process.log_)
pmpm.ALLOW_IP = pmpm.ALLOW_IP.apply(data_process.log_)
pmpm.ALLOW_ER = pmpm.ALLOW_ER.apply(data_process.log_)

CPU times: user 39 s, sys: 436 ms, total: 39.4 s
Wall time: 39.4 s


In [12]:
medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
pmpm['LOB'] = 1
pmpm.loc[medicaid_mask, 'LOB'] = 0
pmpm['MEDICAID_LOB'] = pmpm['LOB']
pmpm = pmpm.drop(['LOB'], axis=1)

In [13]:
pmpm = pmpm[col_seq_cc_all]

## Choose Onset Column Name

In [14]:
onset = "CC_DIABETES"

In [15]:
pmpm.head()

PERS_ID     MYR  AGE_AT_MIDMONTH MBR_GNDR  MEDICAID_LOB  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  201908              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  201909              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  201910              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  201

### Process Canary
1. Drop register dates beyond pmpm max date
2. Transform register date to MYR
3. Keep only ids and MYR

### Get Controls and Treated

In [16]:
%%time
canary2 = process_canary(canary, pmpm.MYR.max())
treated, controls = get_tc(pmpm.copy(), canary2)

CPU times: user 6.71 s, sys: 2.66 s, total: 9.37 s
Wall time: 9.37 s


### Keep Canary Patients from their date of registeration and onwards which did not have CC_DIABETES flag on, on their date of registration

In [17]:
%%time
treated2 = in_canary(treated.copy(), canary2)
treated2 = filter_first_month_diabetes(treated2.copy(), onset=onset)
treated2.drop(columns = ['IN_CANARY'], inplace = True)

CPU times: user 6.85 s, sys: 7.97 ms, total: 6.86 s
Wall time: 6.85 s


### Transform with NN

In [18]:
#!aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Latent_Model_m5 ../../temp/TF_Latent_Model_m5

In [19]:
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')

model = models.load_model(path+"/TF_Latent_Model_m5", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [20]:
cols_to_keep = ["PERS_ID", "MYR", onset]

In [21]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']

In [22]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[col_seq_cc_all].copy(), cols_to_keep)
cntrls = data_process.transform_features_with_NN(model, controls[col_seq_cc_all].copy(), cols_to_keep)

CPU times: user 4min 39s, sys: 1min 7s, total: 5min 46s
Wall time: 2min 7s


### Normalize Original Cols

In [23]:
# cols_to_normalize = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']

# tr2[cols_to_normalize] = tr2[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
# cntrls[cols_to_normalize] = cntrls[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [24]:
# tr2.head()

### For each patient in the treated group in its registartion month find a population of matched controls in the same month by using 

In [25]:
%%time

columns_to_match = latent_cols 

propensity_func_args = {'match_on':columns_to_match, 'neighbors_per_treated': 5}
matched0, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           propensity_func, 
                                           propensity_func_args)

Mathcing Function: propensity_func


Propensity Score Matching...fitting model
Creating Matched Set...


1581it [22:40,  1.16it/s]


CPU times: user 26min 11s, sys: 1min 49s, total: 28min 1s
Wall time: 23min 55s


In [26]:
random_matching_func_args = {'neighbors_per_treated': 100}
random_controls = matched_population(tr2.copy(), 
                                       cntrls.copy(),
                                       random_matching_func, 
                                       random_matching_func_args)

Mathcing Function: random_matching_func


In [28]:
matched_data0.to_pickle(path+'/ldata/match0_l.p')
random_controls.to_pickle(path+'/ldata/random_l.p')
tr2.to_pickle(path+'/ldata/treated0_l.p')

## Transform Back to Original Space - match0, match1, random_controls
matched0 (latent) -> transform using controls -> matched0_orig <br/> 
matched1 (latent) -> transform using controls -> matched1_orig <br/>
random_controls (latent) -> transform using controls -> random_controls_orig <br/>
treated0(original) -> done -> tr0 (latent) <br/>
treated1(original) -> done -> tr1 (latent) <br/>

In [29]:
matched_data0 = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(matched_data0.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])
random_controls = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(random_controls.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])

In [30]:
matched_data0.to_pickle(path+'/odata/match0_o.p')
random_controls.to_pickle(path+'/odata/random_o.p')
treated2.to_pickle(path+'/odata/treated0_o.p')

### Save to AWS

In [31]:
import os
l_files_list = os.listdir(git_path+"/temp/ldata/")
o_files_list = os.listdir(git_path+"/temp/odata/")

#save in S3
import boto3
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'
for file in l_files_list:
    s3.upload_file( git_path+"/temp/ldata/"+file,
                    bucket, 
                    'rpi/nneehal/DATA/ldata/m5_propens_no_cluster_only_latent/'+file,
                    ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})
for file in o_files_list:
    s3.upload_file( git_path+"/temp/odata/"+file,
                    bucket, 
                    'rpi/nneehal/DATA/odata/m5_propens_no_cluster_only_latent/'+file,
                    ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})

## Check Pseudo R2 metric

In [42]:
t_df = tr2.copy()
c_df = cntrls.copy()

In [54]:
t_df["Treated"]=1
c_df["Treated"]=0

In [56]:
df_before_match = pd.concat([t_df, c_df], axis=0)

In [60]:
import pickle
with open(path+'/ldata/match0_l.p', 'rb') as fp:
    m_df = pickle.load(fp)

In [62]:
m_df["Treated"]=0

In [63]:
df_after_match = pd.concat([t_df, m_df], axis=0)

In [68]:
df_before_match[latent_cols]

x1        x2         x3        x4        x5        x6        x7        x8
0        8.697727 -9.663467   9.739600 -6.985378 -3.358981  3.413227 -4.575884  1.595648
1        8.066169 -8.163336   8.775659 -6.213150 -4.665704  2.942258 -3.877832  1.260383
2        8.057712 -8.315437   8.841597 -6.268760 -4.479286  2.899271 -3.873665  1.205960
3        9.002401 -9.905861  10.163186 -7.268509 -3.570164  3.763330 -4.760419  1.758571
4        8.405992 -8.668972   9.335248 -6.610003 -4.593896  3.282717 -4.103991  1.382776
...           ...       ...        ...       ...       ...       ...       ...       ...
8842431  2.811710 -3.526175   3.210663 -1.506433 -1.319809 -1.040877 -2.382749 -1.682494
8842432 -3.668323  3.047153  -6.266740  7.559153  2.360908 -6.261048  0.280549 -5.162400
8842433 -4.775417  6.308419  -5.881477  8.251661  4.277145 -5.144156 -2.372739 -4.206527
8842434 -4.390818  5.437697  -5.842038  7.913243  3.778523 -5.447751 -1.699599 -4.373420
8842435 -4.254379  5.057015  -5.886378  7.789888  3.546260 -5.615122 -1.354576 -4.495584

[8864323 rows x 8 columns]

In [70]:
def efron_rsquare(y, y_pred):
    n = float(len(y))
    t1 = np.sum(np.power(y-y_pred, 2.0))
    t2 = np.sum(np.power((y-np.sum(y)/n), 2.0))
    return 1.0-(t1/t2)

In [74]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=False, solver='lbfgs').fit(df_after_match[latent_cols].values, df_after_match.Treated.values)

y = df_after_match.Treated.values
y_pred = clf.predict_proba(df_after_match[latent_cols].values)[:,1]

In [73]:
efron_rsquare(y, y_pred)

In [75]:
efron_rsquare(y, y_pred)

### Run the Diabetes onset Function

In [ ]:
%%time
con_onset0 = diabet_onset(matched_data0.copy(), 'Diabetes_Con0')

treated_onset0 = diabet_onset(treated2.copy(), 'Diabetes_Tr')

pmpm2 = filter_first_month_diabetes(pmpm.copy())
pmpm_onset = diabet_onset(pmpm2, 'Diabetes_PM')

In [ ]:
random_onset = diabet_onset(random_controls.copy(), 'Diabetes_Rnd')

In [ ]:
con_onset20 = con_onset0.rename(columns = {'Sample': 'Sample_Con0', 'Per_1000':'Per_1000_Con0'})

treated_onset20 = treated_onset0.rename(columns = {'Sample': 'Sample_Tr0', 'Per_1000':'Per_1000_Tr0'})

pmpm_onset2 = pmpm_onset.rename(columns = {'Sample': 'Sample_PM', 'Per_1000':'Per_1000_PM'})

In [ ]:
random_onset2 = random_onset.rename(columns = {'Sample': 'Sample_PM', 'Per_1000':'Per_1000_PM'})
joint = pd.concat((con_onset20, treated_onset20,  pmpm_onset2, random_onset2), axis = 1)

### Diabetes Onset Per Month

1. **_Con : For Matched Controls Population**
2.  **_Tr : For Canary Treated Population**
3. **_PM : For the whole PMPM Population**
4. **_Rnd: For a Random Matching**

In [ ]:
joint.describe().loc[['mean', 'std']]

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-dark')

In [ ]:
joint2 = pd.concat((con_onset0,  treated_onset0,   pmpm_onset, random_onset), axis = 1)
jointcon0 = joint2.Per_1000.iloc[1:, 0].reset_index()
jointTr0 = joint2.Per_1000.iloc[1:, 1].reset_index()
jointpmpm = joint2.Per_1000.iloc[1:, 2].reset_index()
jointrandom = joint2.Per_1000.iloc[1:, 3].reset_index()

**Plot Top Left**: $f(x)$ = Onset Per 1000 per month </br>
**Plot Top Right and Bottom Left**: $F(x) = \int_0^x f(y)dy$ Cumulative sum of the rates

In [ ]:
from diabetes_onset_plotter import plot_onset, plot_onset2
names = ['Matched Controls', 'Treated', 'Random']
f, ax = plot_onset2(jointcon0, jointTr0, jointrandom, names = names)
#f.savefig('figs/onset.png')
plt.show()

In [ ]:
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

#model 4
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR', 'CC_CHRONIC_KIDNEY', 'CC_HEART_FAILURE', 'CC_ISCHEMIC', 
                    'CC_STROKE', 'CC_HYPERLIPIDEMIA', 'CC_HYPERTENSION']

tr_desc0 = treated2[columns_to_match].describe().loc[['mean', 'std']]
tr_desc0 = tr_desc0.T

con_desc0 = matched_data0[columns_to_match].describe().loc[['mean', 'std']]
con_desc0 = con_desc0.T

pmpm_desc = pmpm2[columns_to_match].describe().loc[['mean', 'std']]
pmpm_desc = pmpm_desc.T

random_desc = random_controls[columns_to_match].describe().loc[['mean', 'std']]
random_desc = random_desc.T

alli = pd.concat((tr_desc0, con_desc0, pmpm_desc, random_desc), axis = 1, keys = ['Treated',
                                                                                 'Matched Controls',
                                                                                'Genaral Population', 
                                                                                 'Random'] )

In [ ]:
alli

## Distribution of latent space

In [ ]:
import pickle
with open(path+'/ldata/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata/random_l.p', 'rb') as fp:
    random_latent = pickle.load(fp)
with open(path+'/ldata/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)

In [ ]:
columns_to_match = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']

tr_desc0 = treated_sick_latent[columns_to_match].describe().loc[['mean','std']].T
con_desc0 = controls_sick_latent[columns_to_match].describe().loc[['mean','std']].T
random_desc = random_latent[columns_to_match].describe().loc[['mean','std']].T

alli_latent = pd.concat((tr_desc0, con_desc0, random_desc), axis = 1, keys = ['Treated', 'Matched Controls', 'Random'] )

In [ ]:
alli_latent